In [2]:
%matplotlib widget

import os
import sys
import brainweb
import torch
from brainweb import volshow
import numpy as np
from os import path
from tqdm.auto import tqdm
import logging
import nibabel as nib
import sirf.STIR as stir
msg_red = stir.MessageRedirector('info.txt', 'warnings.txt')
import matplotlib.pyplot as plt
import os
import sirf.Reg as reg
from math import cos, sin, pi, radians
from sirf.Utilities import examples_data_path
import shutil
from tqdm.auto import tqdm
dir_path = os.path.dirname(os.getcwd())
source_path = os.path.join(dir_path, 'source')
sys.path.append(source_path)
from data.ellipses import EllipsesDataset
from reconstruction.reconstruction import *
from reconstruction.registration import *
from reconstruction.osem import *
from plotting_functions import plot_2d_image
import matplotlib.pyplot as plt
import imageio
from plotting_functions import *
from IPython.display import Image
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import matplotlib.pyplot as plt

msg_red = stir.MessageRedirector('info.txt', 'warnings.txt', 'errors.txt')
# load model 
path_to_data = os.path.join(os.path.dirname(os.getcwd()), 'source', 'models')
sys.path.append(path_to_data)

from models import UNet, UNet_2, UNet_og

model = UNet_og.UNet(in_channels=3, n_class=1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

#model_path = '/home/sirfuser/shared/fine_tuned/trans_sens/max/model_m_UNet_n10000_e1000_lr1e-05_b32_s0.8_0.1_r0_gFalse/final_model+202508271011.pth'
#model_path = '/home/sirfuser/shared/fine_tuned/trans_sens/max/model_m_UNet_n10000_e1000_lr1e-05_b32_s0.8_0.1_r0_gFalse/model_fine_tuned_m_UNet_e100_lr1e-05_b16_s0.8_gFalse_norm_max/fine_tuned_model+202509050443.pth'

model_path = '/home/sirfuser/shared/fine_tuned/trans_sens/median/model_m_UNet_n10000_e1000_lr1e-05_b32_s0.8_0.1_r0_gFalse/final_model+202508271300.pth'
#model_path = '/home/sirfuser/shared/fine_tuned/trans_sens/median/model_m_UNet_n10000_e1000_lr1e-05_b32_s0.8_0.1_r0_gFalse/model_fine_tuned_m_UNet_e100_lr1e-05_b16_s0.8_gFalse_norm_median/fine_tuned_model+202509042333.pth'

#model_path = '/home/sirfuser/shared/fine_tuned/trans_sens/mean/model_m_UNet_n10000_e1000_lr1e-05_b32_s0.8_0.1_r0_gFalse/final_model+202509081617.pth'


if os.path.exists(model_path):
    checkpoint = torch.load(model_path, map_location=device) 
    model.load_state_dict(checkpoint['model_state_dict'])      
    print(f"Model loaded from {model_path}")
else:
    print(f"ERROR: Model file not found at {model_path}")



Model loaded from /home/sirfuser/shared/fine_tuned/trans_sens/median/model_m_UNet_n10000_e1000_lr1e-05_b32_s0.8_0.1_r0_gFalse/final_model+202508271300.pth


In [3]:
# Create ellipses dataset for testing
data_path = os.path.join(dir_path, 'data', 'template_data')
emission_image = stir.ImageData(os.path.join(data_path, 'emission.hv'))
attenuation_image = stir.ImageData(os.path.join(data_path, 'attenuation.hv'))
template_sinogram = stir.AcquisitionData(os.path.join(data_path, 'template_sinogram.hs'))

test_data = EllipsesDataset(attenuation_image, template_sinogram, num_samples=100, generate_non_attenuated_sensitivity=False, norm_mode='median')
test_loader = DataLoader(test_data, batch_size=16, shuffle=True)

In [3]:
# Create brainweb dataset for testing
from torch.utils.data import Dataset
import torch
import numpy as np

mu_water = 0.096

class BrainWebTestDataset(Dataset):
    def __init__(self, sens_images, attn_images, emis_images, transformed_attn_images, transformed_sens_images, template_sinogram): 
        self.sens_images = sens_images
        self.attn_images = attn_images
        self.emis_images = emis_images
        self.transformed_attn_images = transformed_attn_images
        self.transformed_sens_images = transformed_sens_images
        self.template_sinogram = template_sinogram

    def __len__(self):
        return len(self.sens_images)

    def __getitem__(self, idx):
        sens = self.sens_images[idx]
        attn = self.attn_images[idx]
        emis = self.emis_images[idx]
        transformed_attn = self.transformed_attn_images[idx]
        transformed_sens = self.transformed_sens_images[idx]
        
        # Get data as arrays
        sens_array = sens.as_array()
        attn_array = attn.as_array()
        trans_attn_array = transformed_attn.as_array()
        trans_sens_array = transformed_sens.as_array()

        
        # Create non-attenuated sensitivity images for normalisation
        acq_model_no_attn = stir.AcquisitionModelUsingRayTracingMatrix()
        acq_model_no_attn.set_num_tangential_LORs(10)
        acq_model_no_attn.set_up(template_sinogram, emis)
        uniform_acq_data = template_sinogram.get_uniform_copy(1.0)
        non_att = acq_model_no_attn.backward(uniform_acq_data)
        non_att_array = non_att.as_array()
        #denom = float(non_att_array.max()) + 1e-8 # normalise by max of non-attenuated sensitivity image
        #denom = np.mean(non_att_array) + 1e-8 # normalise by mean of non_attenuated sensitivity image
        denom = np.median(non_att_array) + 1e-8 # normalise by median of non-attenuated sensitvity image

        # Normalise data
        sens_norm = sens_array/denom
        attn_norm = attn_array/mu_water # attenuation images normalised by the attenuation of water 
        trans_attn_norm = trans_attn_array/mu_water
        trans_sens_norm = trans_sens_array/denom

        x_sens = sens_norm.squeeze()
        x_attn = attn_norm.squeeze()
        x_attn_trans = trans_attn_norm.squeeze()
        x = np.array([x_sens, x_attn, x_attn_trans]) # inputs 
    
        y = trans_sens_norm # target

        # Build target
        y = torch.tensor(trans_sens_norm, dtype=torch.float32)
        if y.ndim == 2:   # [H,W] → make it [1,H,W]
            y = y.unsqueeze(0)
        assert y.ndim == 3 and y.shape[0] == 1, f"Bad y shape: {y.shape}"

        return (
            torch.tensor(x, dtype=torch.float32),
            y,
            torch.tensor(denom, dtype=torch.float32)
        )



In [4]:
import torch
import numpy as np

dir_path = os.path.dirname(os.getcwd())
#test_dir = '/home/sirfuser/devel/PET_sensitvity'
test_dir = '/home/sirfuser/shared/ft_test/'
template_path = os.path.join(dir_path, 'data', 'template_data')
template_sinogram = stir.AcquisitionData(os.path.join(template_path, 'template_sinogram.hs'))


sens_images = []
attn_images = []
emis_images = []
transformed_attn_images = []
transformed_sens_images = []

num_subjects = [2, 6, 11, 17]
num_slices = 5


for i in num_subjects:
    for j in range(1, num_slices + 1):
        sens = stir.ImageData(os.path.join(test_dir, f'sens_{i}_{j}.hv'))
        attn = stir.ImageData(os.path.join(test_dir, f'attn_{i}_{j}.hv'))
        emis = stir.ImageData(os.path.join(test_dir, f'emission_{i}_{j}.hv'))
        transformed_attn = stir.ImageData(os.path.join(test_dir, f'transform_attn_{i}_{j}.hv'))
        transformed_sens = stir.ImageData(os.path.join(test_dir, f'transform_sens_{i}_{j}.hv'))
        sens_images.append(sens)
        attn_images.append(attn)
        emis_images.append(emis)
        transformed_attn_images.append(transformed_attn)
        transformed_sens_images.append(transformed_sens)

test_dataset = BrainWebTestDataset(sens_images, attn_images, emis_images, transformed_attn_images, transformed_sens_images, template_sinogram)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
# Test model and save or show model output plots

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import os

def compute_rmse(pred, target):
    """Compute RMSE between two tensors."""
    return torch.sqrt(torch.mean((pred - target) ** 2)).item()

# Compute global RMSE over the whole test set
total_rmse = 0.0
count = 0

model.eval()
with torch.no_grad():
    for X, y, denom in test_loader:
        X, y, denom = X.to(device), y.to(device), denom.to(device)
        out = model(X)   # Model predicts transformed sens (normalised)

        # Denormalise
        pred_trans = out * denom.view(-1, 1, 1)       # Predicted transformed sens
        true_trans = y.squeeze(1) * denom.view(-1, 1, 1)  # Ground truth transformed sens

        # Per-image RMSE
        batch_rmse = torch.sqrt(
            F.mse_loss(pred_trans, true_trans, reduction="none")
            .view(pred_trans.size(0), -1)
            .mean(dim=1)
        )
        total_rmse += batch_rmse.sum().item()
        count += batch_rmse.numel()

global_rmse = total_rmse / count
print(f"Global Test Set RMSE: {global_rmse:.6f}")

# Plot one slice per subject
slices_per_subject = 5  
slice_choice = "middle"   # "first", "middle", "last", or int

# Create figure of plots
n_subjects = len(num_subjects)
fig, axes = plt.subplots(n_subjects, 4, figsize=(16, 4 * n_subjects))

if n_subjects == 1:  # ensure axes is 2D array even for 1 subject
    axes = axes[None, :]

with torch.no_grad():
    for subj_idx, subj_number in enumerate(num_subjects):  # num_subjects = [2, 6, 11, 17]
        # Pick slice index for this subject
        if slice_choice == "first":
            dataset_idx = subj_idx * slices_per_subject
        elif slice_choice == "middle":
            dataset_idx = subj_idx * slices_per_subject + slices_per_subject // 2
        elif slice_choice == "last":
            dataset_idx = subj_idx * slices_per_subject + (slices_per_subject - 1)
        else:
            dataset_idx = subj_idx * slices_per_subject + int(slice_choice)

        slice_num = (dataset_idx % slices_per_subject) + 1

        # Load slice
        X, y, denom = test_dataset[dataset_idx]
        X = X.unsqueeze(0).to(device)   # add batch dim
        y = y.unsqueeze(0).to(device)
        denom = denom.unsqueeze(0).to(device)

        # Forward pass
        out = model(X)

        # Denormalise
        orig_sens = X[:, 0, :, :] * denom.view(-1, 1, 1)     # original sensitivity
        pred_trans = out * denom.view(-1, 1, 1)              # predicted transformed sens
        true_trans = y.squeeze(1) * denom.view(-1, 1, 1)     # ground truth transformed sens

        # Convert to numpy
        orig_np = orig_sens[0].detach().cpu().numpy().squeeze()
        pred_np = pred_trans[0].detach().cpu().numpy().squeeze()
        true_np = true_trans[0].detach().cpu().numpy().squeeze()
        diff_map = pred_np - true_np

        # RMSE for this slice
        slice_rmse = compute_rmse(pred_trans, true_trans)

        # Fixed scales (use global values to keep comparable)
        vmin, vmax = 0, 13
        diff_abs_max = max(abs(diff_map.min()), abs(diff_map.max()))

         # plots for this subject row
        ax1, ax2, ax3, ax4 = axes[subj_idx]

        im1 = ax1.imshow(orig_np, cmap="viridis", vmin=vmin, vmax=vmax)
        ax1.set_title("Original Sensitivity")
        ax1.axis("off")
        fig.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)

        im2 = ax2.imshow(pred_np, cmap="viridis", vmin=vmin, vmax=vmax)
        ax2.set_title("Predicted Output")
        ax2.axis("off")
        fig.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)

        im3 = ax3.imshow(true_np, cmap="viridis", vmin=vmin, vmax=vmax)
        ax3.set_title("Transformed Sensitivity (Target)")
        ax3.axis("off")
        fig.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)

        im4 = ax4.imshow(diff_map, cmap="bwr", vmin=-2, vmax=2)
        ax4.set_title("Difference Map (Output - Target)")
        ax4.axis("off")
        fig.colorbar(im4, ax=ax4, fraction=0.046, pad=0.04, label="Difference")

        row_axes = [ax1, ax2, ax3, ax4]
        row_pos = row_axes[0].get_position()
        y_top = row_pos.y1
        fig.text(
            0.5, y_top + 0.02,
            f"Subject {subj_number}, Slice {slice_num} - "
            f"Slice RMSE={slice_rmse:.4f}, Global RMSE={global_rmse:.4f}",
            ha="center", va="bottom", fontsize=12
        )

# Add one global title
plt.suptitle("Transformed Sensitivity Fine-Tuned Model Outputs (Median Normalisation)", fontsize=16, y=0.93)
#plt.show()
# Save figure
plt.savefig("trans_sens_ft_median_results.png", dpi=300, bbox_inches="tight")
plt.close()
print('Figure Saved')

In [5]:
# Save model outputs for reconstruction

import sirf.STIR as stir
import numpy as np
import os
import torch

save_dir = os.path.join(dir_path, "data", "file")
os.makedirs(save_dir, exist_ok=True)

num_slices = 5
test_subjects = [2, 6, 11, 17]

model.eval()
with torch.no_grad():
    for idx, (X, y, denom) in enumerate(test_loader):
        X, y, denom = X.to(device), y.to(device), denom.to(device)
        out = model(X) 

        # denormalise
        pred_trans = out * denom.view(-1, 1, 1, 1) 

        for b in range(pred_trans.size(0)):
            global_idx = idx * test_loader.batch_size + b

            subject = test_subjects[global_idx // num_slices]
            slice_num = (global_idx % num_slices) + 1

            # take one slice
            pred_np = pred_trans[b, 0].detach().cpu().numpy()
            pred_np = pred_np[np.newaxis, :, :]  

            # fill into template image
            template = test_loader.dataset.sens_images[global_idx]
            pred_image = template.clone()
            pred_image.fill(pred_np)

            # save
            filename = f"pred_sens_{subject}_{slice_num}.hv"
            filepath = os.path.join(save_dir, filename)
            pred_image.write(filepath)

            # check data aligns
            try:
                template_name = os.path.basename(template.filename())
            except AttributeError:
                template_name = "<no filename available>"

            print(
                f"global_idx={global_idx:2d} | expected: sens_{subject}_{slice_num}.hv "
                f"| template: {template_name} | saved: {filename}"
            )


global_idx= 0 | expected: sens_2_1.hv | template: <no filename available> | saved: pred_sens_2_1.hv
global_idx= 1 | expected: sens_2_2.hv | template: <no filename available> | saved: pred_sens_2_2.hv
global_idx= 2 | expected: sens_2_3.hv | template: <no filename available> | saved: pred_sens_2_3.hv
global_idx= 3 | expected: sens_2_4.hv | template: <no filename available> | saved: pred_sens_2_4.hv
global_idx= 4 | expected: sens_2_5.hv | template: <no filename available> | saved: pred_sens_2_5.hv
global_idx= 5 | expected: sens_6_1.hv | template: <no filename available> | saved: pred_sens_6_1.hv
global_idx= 6 | expected: sens_6_2.hv | template: <no filename available> | saved: pred_sens_6_2.hv
global_idx= 7 | expected: sens_6_3.hv | template: <no filename available> | saved: pred_sens_6_3.hv
global_idx= 8 | expected: sens_6_4.hv | template: <no filename available> | saved: pred_sens_6_4.hv
global_idx= 9 | expected: sens_6_5.hv | template: <no filename available> | saved: pred_sens_6_5.hv
